In [11]:
import cracknuts as cn

In [2]:
g1 = cn.cracker_g1('192.168.0.10')
g1.connect(force_update_bin=True, force_write_default_config=True)

In [3]:
import random
import time
from cracknuts.cracker import serial


cmd_set_aes_enc_key = "01 00 00 00 00 00 00 10"
cmd_aes_enc = "01 02 00 00 00 00 00 10"

aes_key = "11 22 33 44 55 66 77 88 99 00 aa bb cc dd ee ff"
aes_data_len = 16

sample_length = 20000

def init(cracker):
    
    cracker.nut_clock_freq('8M')
    cracker.nut_clock_enable()
    cracker.nut_voltage(3.3)
    cracker.nut_voltage_enable()
    cracker.uart_io_enable()
    cracker.osc_sample_clock('48m')
    cracker.osc_sample_length(sample_length)
    cracker.osc_trigger_source('N')
    # cracker.osc_analog_enable('A')
    # cracker.osc_analog_gain('A', 1)
    # cracker.osc_analog_disable('B')
    cracker.osc_analog_gain('B', 10)
    cracker.osc_trigger_level(0)
    cracker.osc_trigger_mode('E')
    cracker.osc_trigger_edge('U')
    cracker.uart_config(
        baudrate=serial.Baudrate.BAUDRATE_115200, 
        bytesize=serial.Bytesize.EIGHTBITS, 
        parity=serial.Parity.PARITY_NONE, 
        stopbits=serial.Stopbits.STOPBITS_ONE)

    time.sleep(2)
    cmd = cmd_set_aes_enc_key + aes_key
    status, ret = cracker.uart_transmit_receive(cmd, timeout=1000, rx_count=6)

def do(cracker, count):
    # plaintext_data = random.randbytes(aes_data_len)
    plaintext_data = bytes.fromhex('AA BB CC DD EE FF 00 11 22 33 44 55 66 77 88 99')
    tx_data = bytes.fromhex(cmd_aes_enc.replace(' ', '')) + plaintext_data
    status, ret = cracker.uart_transmit_receive(tx_data, rx_count= 6 + aes_data_len, is_trigger=True)
    print(ret.hex(' ') if ret else 'None')
    return {
        "plaintext": plaintext_data,
        "ciphertext": b'',
        "key": bytes.fromhex(aes_key)
    }


acq = cn.simple_glitch_acq(g1, init, do)

p = cn.show_panel(acq)
p

In [8]:
g1.glitch_vcc_normal(3.3)
g1.glitch_gnd_normal(0.0)
g1.glitch_vcc_config(wait=1, level=1.5, length=20, delay=10, repeat=2)
g1.nut_voltage_enable()

(0, None)

In [10]:
g1.glitch_vcc_force()

(0, None)

In [ ]:
g1.glitch_vcc_normal(3.3)
g1.glitch_gnd_normal(0.0)
g1.glitch_gnd_config(wait=1, level=1.5, length=20, delay=10, repeat=2)
g1.nut_voltage_enable()

In [ ]:
g1.glitch_gnd_force()

In [ ]:
g1.nut_clock_freq('8m')
# 实际配置的是
# g1.glitch_clock_normal([
#     3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2, 3.2,
#     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
# ])

# 配置一个10m的时钟glitch
g1.glitch_clock_config(
    wave=[
        1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5,
        0, 0, 0, 0, 0, 0, 0, 0,
    ],
    wait=10,
    delay=5,
    repeat=3
)
g1.nut_clock_enable()
g1.nut_voltage_enable()

In [ ]:
g1.glitch_clock_force()

In [ ]:
# 配置一个10m的时钟glitch
g1.glitch_clock_config(
    wave=[
        3.2, 3.2, 3.2, 3.2,
        0.0, 0.0, 0.0, 0.0,
        3.2, 3.2, 3.2, 3.2,
        0.0, 0.0, 0.0, 0.0,
        3.2, 3.2, 3.2, 3.2,
        0.0, 0.0, 0.0, 0.0,
        3.2, 3.2, 3.2, 3.2,
        0.0, 0.0, 0.0, 0.0,
    ],
    wait=10,
    delay=5,
    repeat=3
)
 

In [7]:
g1.get_current_config()

Config(osc_channel_0_enable: False, osc_channel_1_enable: True, osc_channel_0_gain: 10, osc_channel_1_gain: 10, osc_sample_length: 1024, osc_sample_delay: 0, osc_sample_clock: 48000, osc_sample_phase: 0, osc_trigger_source: 0, osc_trigger_mode: 0, osc_trigger_edge: 0, osc_trigger_edge_level: 1, nut_enable: True, nut_voltage: 3.3, nut_clock_enable: True, nut_clock: 8000, nut_timeout: None, nut_reset_io_enable: False, nut_uart_enable: False, nut_uart_baudrate: Baudrate.BAUDRATE_115200, nut_uart_bytesize: Bytesize.EIGHTBITS, nut_uart_parity: Parity.PARITY_NONE, nut_uart_stopbits: Stopbits.STOPBITS_ONE, nut_spi_enable: False, nut_spi_speed: 5000, nut_spi_cpol: SpiCpol.SPI_CPOL_LOW, nut_spi_cpha: SpiCpha.SPI_CPHA_LOW, nut_spi_csn_auto: True, nut_spi_csn_delay: True, nut_i2c_enable: False, nut_i2c_dev_addr: 0, nut_i2c_speed: I2cSpeed.STANDARD_100K, nut_i2c_stretch_enable: False, glitch_vcc_arm: False, glitch_vcc_normal: 3.3, glitch_vcc_config_wait: 9, glitch_vcc_config_level: 2.0, glitch_vcc

In [ ]:
import struct

struct.unpack(">I", g1.register_read(base_address=0x43c10000, offset=0x150)[1])

In [ ]:
from cracknuts.cracker.cracker_g1 import _vcc_interp_func_cv

g1._get_voltage_from_dac_code(671, _vcc_interp_func_cv)

In [ ]:
g1.get_default_config()